# Smart Logistics Optimizer
An interactive system for optimizing delivery routes and logistics operations.

In [2]:
# Install required packages
!pip install numpy pandas scikit-learn folium ipywidgets geopy ipyleaflet

In [3]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipyleaflet import Map, Marker, Polyline, Icon
from geopy.distance import geodesic
import folium
import json

# Enable widgets
from ipywidgets import Layout, Box, Label

In [4]:
class LogisticsOptimizer:
    def __init__(self):
        self.scaler = StandardScaler()
        self.kmeans = KMeans(n_clusters=5, random_state=42)
        self.delivery_points = []
        self.setup_ui()
    
    def setup_ui(self):
        # Create input widgets
        style = {'description_width': '100px'}
        layout = Layout(width='300px')
        
        self.lat_input = widgets.FloatText(
            description='Latitude:',
            style=style,
            layout=layout
        )
        
        self.lng_input = widgets.FloatText(
            description='Longitude:',
            style=style,
            layout=layout
        )
        
        self.weight_input = widgets.FloatText(
            description='Weight (kg):',
            style=style,
            layout=layout
        )
        
        # Create buttons
        self.add_button = widgets.Button(
            description='Add Point',
            button_style='primary',
            layout=Layout(width='150px')
        )
        
        self.optimize_button = widgets.Button(
            description='Optimize Routes',
            button_style='success',
            layout=Layout(width='150px')
        )
        
        self.clear_button = widgets.Button(
            description='Clear All',
            button_style='danger',
            layout=Layout(width='150px')
        )
        
        # Create output areas
        self.points_output = widgets.Output()
        self.results_output = widgets.Output()
        
        # Create map
        self.map = Map(center=(20.5937, 78.9629), zoom=5)
        
        # Add event handlers
        self.add_button.on_click(self.add_point)
        self.optimize_button.on_click(self.optimize_routes)
        self.clear_button.on_click(self.clear_all)
        
        # Layout the UI
        input_box = Box([
            self.lat_input, 
            self.lng_input, 
            self.weight_input,
            widgets.HBox([self.add_button, self.optimize_button, self.clear_button])
        ])
        
        display(HTML('<h3>Add Delivery Points</h3>'))
        display(input_box)
        display(HTML('<h3>Delivery Points</h3>'))
        display(self.points_output)
        display(HTML('<h3>Results</h3>'))
        display(self.results_output)
        display(HTML('<h3>Map</h3>'))
        display(self.map)
    
    def add_point(self, b):
        point = {
            'lat': self.lat_input.value,
            'lng': self.lng_input.value,
            'weight': self.weight_input.value
        }
        
        self.delivery_points.append(point)
        
        # Add marker to map
        marker = Marker(location=(point['lat'], point['lng']))
        self.map.add_layer(marker)
        
        # Update points list
        with self.points_output:
            clear_output()
            for i, p in enumerate(self.delivery_points):
                print(f"Point {i+1}: ({p['lat']}, {p['lng']}) - {p['weight']}kg")
        
        # Clear inputs
        self.lat_input.value = 0.0
        self.lng_input.value = 0.0
        self.weight_input.value = 0.0
    
    def optimize_routes(self, b):
        if len(self.delivery_points) < 2:
            with self.results_output:
                clear_output()
                print("Please add at least 2 delivery points")
            return
        
        # Convert points to numpy array
        points = np.array([[p['lat'], p['lng']] for p in self.delivery_points])
        
        # Perform clustering
        clusters = self.kmeans.fit_predict(points)
        
        # Group points by cluster
        routes = {}
        for i, cluster in enumerate(clusters):
            if cluster not in routes:
                routes[cluster] = []
            routes[cluster].append(self.delivery_points[i])
        
        # Calculate costs and visualize routes
        colors = ['red', 'blue', 'green', 'purple', 'orange']
        
        # Clear previous routes
        self.map.clear_layers()
        
        with self.results_output:
            clear_output()
            print("Optimized Routes:")
            
            for cluster_id, points in routes.items():
                color = colors[cluster_id % len(colors)]
                
                # Calculate route cost
                total_distance = 0
                prev_point = None
                coordinates = []
                
                for point in points:
                    coordinates.append((point['lat'], point['lng']))
                    if prev_point:
                        distance = geodesic(
                            (prev_point['lat'], prev_point['lng']),
                            (point['lat'], point['lng'])
                        ).kilometers
                        total_distance += distance
                    prev_point = point
                    
                    # Add marker
                    marker = Marker(location=(point['lat'], point['lng']))
                    self.map.add_layer(marker)
                
                # Add route line
                line = Polyline(
                    locations=coordinates,
                    color=color,
                    weight=2,
                    opacity=0.8
                )
                self.map.add_layer(line)
                
                # Calculate cost
                cost = total_distance * 2.5  # Example rate
                
                print(f"Route {cluster_id + 1}:")
                print(f"  - Stops: {len(points)}")
                print(f"  - Distance: {total_distance:.2f} km")
                print(f"  - Cost: ${cost:.2f}")
                print()
    
    def clear_all(self, b):
        self.delivery_points = []
        self.map.clear_layers()
        
        with self.points_output:
            clear_output()
        
        with self.results_output:
            clear_output()

In [5]:
# Create and display the optimizer
optimizer = LogisticsOptimizer()

Box(children=(FloatText(value=0.0, description='Latitude:', layout=Layout(width='300px'), style=DescriptionSty…

Output()

Output()

Map(center=[20.5937, 78.9629], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

## Sample Usage

Try these sample coordinates:

1. Mumbai: (19.0760, 72.8777)
2. Delhi: (28.6139, 77.2090)
3. Bangalore: (12.9716, 77.5946)
4. Chennai: (13.0827, 80.2707)
5. Hyderabad: (17.3850, 78.4867)